In [10]:
import pandas as pd
import numpy as np

gu_change=pd.read_csv('data/15.상권분석서비스(자치구별 상권변화지표).csv', encoding='utf-8')
people=pd.read_csv('all-people.csv' ,encoding='utf-8')
df0=pd.read_csv('data/00.상권분석서비스(상권-영역).csv',encoding='utf-8')

# df0

In [12]:
gu_name=gu_change[['시군구_코드','시군구_코드_명']].drop_duplicates()
# gu_name

gu_t1=gu_change[['시군구_코드','운영_영업_개월_평균', '폐업_영업_개월_평균', '상권_변화_지표_명']]
gu_t1=gu_t1.replace('상권확장',1).replace('다이나믹' ,4).replace('정체',2).replace('상권축소',3)

# people.info()
# people[people.columns[1:]]=people[people.columns[1:]].astype('int64')
df0.columns
cd_df=df0[[ '상권_코드', '시군구_코드']]
people=people.merge(cd_df, on='상권_코드').drop_duplicates()

In [15]:
pop_gp = people.groupby(['시군구_코드','기준_년_분기_코드']).sum().reset_index()

pop_gp['주중_유동인구_비율']=(pop_gp['주중_유동인구_수']/(pop_gp['주중_유동인구_수']+pop_gp['주말_유동인구_수'])*100).round(0)
pop_gp['주말_유동인구_비율']=(pop_gp['주말_유동인구_수']/(pop_gp['주중_유동인구_수']+pop_gp['주말_유동인구_수'])*100).round(0)

pop_gp_1=pop_gp[['시군구_코드','기준_년_분기_코드','주중_유동인구_비율','주말_유동인구_비율']]

In [17]:
gu_change

,기준_년_분기_코드,시군구_코드,시군구_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2020_1,11740,강동구,LL,다이나믹,106,50,110,54
1,2020_1,11710,송파구,LH,상권확장,108,54,110,54
2,2020_1,11680,강남구,LL,다이나믹,102,50,110,54
3,2020_1,11650,서초구,HH,정체,118,54,110,54
4,2020_1,11620,관악구,LL,다이나믹,108,52,110,54
...,...,...,...,...,...,...,...,...,...
633,2014_1,11215,광진구,LL,다이나믹,89,44,91,44
634,2014_1,11200,성동구,HH,정체,93,46,91,44
635,2014_1,11170,용산구,HH,정체,101,47,91,44
636,2014_1,11140,중구,HH,정체,114,49,91,44


In [ ]:
pop_gp_1.merge()

In [4]:
gu_t1.rename(columns={'시군구_코드':'guCode', '운영_영업_개월_평균':'openMonthMean', '폐업_영업_개월_평균':'closeMonthMean','상권_변화_지표_명':'guChangPoint'})

,guCode,openMonthMean,closeMonthMean,guChangPoint
0,11740,106,50,4
1,11710,108,54,1
2,11680,102,50,4
3,11650,118,54,2
4,11620,108,52,4
...,...,...,...,...
633,11215,89,44,4
634,11200,93,46,2
635,11170,101,47,2
636,11140,114,49,2


In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [9]:
# X=gu_t1.iloc[:,:-1]
# y=gu_t1.iloc[:,-1:]



X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.3, 
                                                    stratify=y,
                                                    random_state=42)

In [11]:
# DecisionTreeClassifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=42)

# 학습수행
dt_clf.fit(X_train, y_train)

print(dt_clf.score(X_train, y_train), dt_clf.score(X_test, y_test))
# 학습이 완료된 DecisionTreeClassifier 객체에서 테스트 데이터 세트로 예측 수행
y_pred = dt_clf.predict(X_test)

from sklearn.metrics import accuracy_score
print('예측정확도 : {0: .4f}'.format(accuracy_score(y_test, y_pred)))

from sklearn.model_selection import cross_val_score

cross_val_score(dt_clf, X_test, y_test, cv=10).mean()

0.99581589958159 0.93125
예측정확도 :  0.9313


0.9125

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 학습 진행
forest = RandomForestClassifier(n_estimators=100, max_depth=10)
forest.fit(X_train, y_train)

# 예측
y_pred = forest.predict(X_test)
print(y_pred)
print(list(y_test))

#Random forest 정확도 측정
print(forest.score(X_train, y_train),forest.score(X_test, y_test))


# 정확도 확인
# accuracy_score(실제 레이블 세트, 예측 레이블 세트) : 정확도를 측정하기 위한 사이킷런의 함수
print('정확도 :', accuracy_score(y_test, y_pred))

from sklearn.model_selection import cross_val_score
cross_val_score(forest, X_test, y_test, cv=10).mean()

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[4 4 3 3 4 4 4 4 4 4 4 4 4 4 4 2 2 4 1 4 4 4 2 2 4 2 4 3 2 2 4 4 4 4 1 2 4
 4 4 4 1 4 4 4 4 2 3 2 4 4 2 4 4 2 2 4 3 2 2 2 4 4 4 2 2 4 4 3 4 2 4 4 4 2
 4 2 4 4 2 2 3 4 3 4 3 4 4 2 2 2 2 4 4 1 1 4 3 4 2 4 2 4 2 4 2 4 1 2 3 4 4
 2 4 2 4 3 4 2 2 2 2 4 4 2 4 4 1 4 4 2 2 2 2 4 2 4 4 4 3 2 4 4 3 4 2 4 4 4
 4 4 4 2 4 2 2 4 3 4 4 1]
['상권_변화_지표_명']
0.9937238493723849 0.95
정확도 : 0.95


c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\model_selection\_validation.py:

0.85625

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=4,max_depth=10)
gbc.fit(X_train, y_train)

print(gbc.score(X_train, y_train), gbc.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
cross_val_score(gbc, X_test, y_test, cv=10).mean()

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

0.99581589958159 0.94375


c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\programdata\miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

0.84375